In [1]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter07'

In [2]:
import numpy as np
import tensorflow as tf
import os

from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.datasets import imdb

In [3]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

csv_test_dir_prefix = 'imdb_data/test'
csv_test_filename = 'test.csv'
csv_test_dir = os.path.join(os.getcwd(), csv_test_dir_prefix)
os.makedirs(csv_test_dir, exist_ok=True)

np.savetxt(os.path.join(csv_test_dir, csv_test_filename), 
           np.array(x_test[:1000], dtype=np.int32), fmt='%d', delimiter=",")

25000 train sequences
25000 test sequences
x_train shape: (25000,)
x_test shape: (25000, 400)


In [3]:
from sagemaker.tensorflow import TensorFlow

training_job_name='imdb-tf-2021-09-21-17-37-20'

attached_estimator = TensorFlow.attach(training_job_name)
attached_estimator.logs()


2021-09-21 17:45:40 Starting - Preparing the instances for training
2021-09-21 17:45:40 Downloading - Downloading input data
2021-09-21 17:45:40 Training - Training image download completed. Training in progress.
2021-09-21 17:45:40 Uploading - Uploading generated training model
2021-09-21 17:45:40 Completed - Training job completed
2021-09-21 17:45:40 Starting - Preparing the instances for training
2021-09-21 17:45:40 Downloading - Downloading input data
2021-09-21 17:45:40 Training - Training image download completed. Training in progress.
2021-09-21 17:45:40 Uploading - Uploading generated training model
2021-09-21 17:45:40 Completed - Training job completed2021-09-21 17:42:03,305 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-09-21 17:42:03,901 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test"

In [6]:
attached_estimator.model_data

's3://sagemaker-us-west-2-552106442228/imdb-tf-2021-09-21-17-37-20/output/model.tar.gz'

In [8]:
predictor = attached_estimator.deploy(initial_instance_count=1, 
                                      instance_type='ml.c5.xlarge',
                                      wait=True)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


------------!

In [41]:
predictor.endpoint_name

'imdb-tf-2021-09-21-17-37-20-2021-10-04-21-28-40-974'

In [5]:
sagemaker_client = sess.boto_session.client('sagemaker')
autoscaling_client = sess.boto_session.client('application-autoscaling')

endpoint_name = 'imdb-tf-2021-09-21-17-37-20-2021-10-04-21-28-40-974'
#predictor.endpoint_name
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
print(response)

{'EndpointName': 'imdb-tf-2021-09-21-17-37-20-2021-10-04-21-28-40-974', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:552106442228:endpoint/imdb-tf-2021-09-21-17-37-20-2021-10-04-21-28-40-974', 'EndpointConfigName': 'imdb-tf-2021-09-21-17-37-20-2021-10-04-21-28-40-974', 'ProductionVariants': [{'VariantName': 'AllTraffic', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-inference:2.1-cpu', 'ResolvedImage': '763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-inference@sha256:0803dbfa124acf1d43f1493f84d55a435e5894bbdd839fdefe20c750e815bca0', 'ResolutionTime': datetime.datetime(2021, 10, 4, 21, 28, 46, 962000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 2, 'DesiredInstanceCount': 2}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2021, 10, 4, 21, 28, 41, 572000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2021, 10, 4, 23, 27, 21, 406000, tzinfo=tzlocal()), 'Respons

In [8]:
resource_id=f'endpoint/{endpoint_name}/variant/AllTraffic' # This is the format in which application autoscaling references the endpoint

response = autoscaling_client.register_scalable_target(
    ServiceNamespace='sagemaker', #
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=1,
    MaxCapacity=4
)

#Example 1 - SageMakerVariantInvocationsPerInstance Metric
response = autoscaling_client.put_scaling_policy(
    PolicyName='Invocations-ScalingPolicy',
    ServiceNamespace='sagemaker', # The namespace of the AWS service that provides the resource. 
    ResourceId=resource_id, # Endpoint name 
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', # SageMaker supports only Instance Count
    PolicyType='TargetTrackingScaling', # 'StepScaling'|'TargetTrackingScaling'
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 2000.0, # The target value for the metric. - here the metric is - SageMakerVariantInvocationsPerInstance
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance', # is the average number of times per minute that each instance for a variant is invoked. 
        },
        'ScaleInCooldown': 600, # The cooldown period helps you prevent your Auto Scaling group from launching or terminating 
                                # additional instances before the effects of previous activities are visible. 
                                # You can configure the length of time based on your instance startup time or other application needs.
                                # ScaleInCooldown - The amount of time, in seconds, after a scale in activity completes before another scale in activity can start. 
        'ScaleOutCooldown': 300 # ScaleOutCooldown - The amount of time, in seconds, after a scale out activity completes before another scale out activity can start.
        
        # 'DisableScaleIn': True|False - ndicates whether scale in by the target tracking policy is disabled. 
                            # If the value is true , scale in is disabled and the target tracking policy won't remove capacity from the scalable resource.
    }
)

In [9]:
response = autoscaling_client.describe_scaling_policies(
    ServiceNamespace='sagemaker'
)
# print(response)
for i in response['ScalingPolicies']:
    print('')
    print(i['PolicyName'])
    print('')
    if('TargetTrackingScalingPolicyConfiguration' in i):
        print(i['TargetTrackingScalingPolicyConfiguration']) 
    else:
        print(i['StepScalingPolicyConfiguration'])
    print('')


Invocations-ScalingPolicy

{'TargetValue': 300.0, 'PredefinedMetricSpecification': {'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'}, 'ScaleOutCooldown': 300, 'ScaleInCooldown': 600}



In [13]:
def get_sentiment(score):
    return 'positive' if score > 0.5 else 'negative' 

In [14]:
import re

regex = re.compile(r'^[\?\s]+')
word_index = imdb.get_word_index()

In [15]:
data_index=199

In [18]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
first_decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') 
                                 for i in x_test[data_index]])
regex.sub('', first_decoded_review)

"i watched this movie purely for the setting it was filmed in an old hotel that a friend owns shares of the plot was predictable the acting was ? at best the scares were all gross outs not true scares br br i don't remember much of the plot and i think that's because there wasn't much of one to remember they didn't even use the hotel to it's fullest potential the beaches are fantastic and the hotel is situated on a ? at low tide you can walk almost 1 4 mile into the bay which is actually an eerie sight first thing in the morning or late at night when the wind is howling through the cracks br br the best way to see this movie is with the remote in your hand so you can fast forward through the action and i'm using that term ? scenes and pause at the beauty of the surroundings"

In [28]:
prediction=predictor.predict(x_test[data_index])

print(f'Labeled sentiment for this review is {get_sentiment(y_test[data_index])}')
print(f'Predicted sentiment is {get_sentiment(prediction["predictions"][0][0])}')

Labeled sentiment for this review is negative
Predicted sentiment is negative


In [16]:
predictor.predict(x_test[data_index])

{'predictions': [[1.80986511e-11]]}

In [17]:
predictor.predict(x_test[:10])

{'predictions': [[0.00371244829],
  [1.0],
  [1.0],
  [0.400452465],
  [1.0],
  [1.0],
  [0.163813606],
  [0.10115058],
  [0.793149233],
  [1.0]]}

In [ ]:
#Example 2 - CPUUtilization metric
response = autoscaling_client.put_scaling_policy(
    PolicyName='CPUUtil-ScalingPolicy',
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 90.0,
        'CustomizedMetricSpecification':
        {
            'MetricName': 'CPUUtilization',
            'Namespace': '/aws/sagemaker/Endpoints',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name },
                {'Name': 'VariantName','Value': 'AllTraffic'}
            ],
            'Statistic': 'Average', # Possible - 'Statistic': 'Average'|'Minimum'|'Maximum'|'SampleCount'|'Sum'
            'Unit': 'Percent'
        },
        'ScaleInCooldown': 600,
        'ScaleOutCooldown': 300
    }
)